In [1]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.0 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY_DEFAULT")  # paste your key
print(" API key set.")

 API key set.


In [6]:
!pip install -qU langchain-classic

In [75]:
import ast
import operator as op
from typing import Union

from langchain_core.tools import tool
import langchain, langchain_core
from langchain_google_genai import ChatGoogleGenerativeAI       # to work with Gemini models
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor, create_react_agent

# Custom tool description renderer
def custom_render_text_description(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])


 # Allowed operators
_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.FloorDiv: op.floordiv, ast.Mod: op.mod,
    ast.Pow: op.pow, ast.UAdd: op.pos, ast.USub: op.neg,
}

def _eval_node(node) -> Union[int, float]:
    # Python 3.8+ uses Constant for numbers
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _OPS:
        return _OPS[type(node.op)](_eval_node(node.left), _eval_node(node.right))
    if isinstance(node, ast.UnaryOp) and type(node.op) in _OPS:
        return _OPS[type(node.op)](_eval_node(node.operand))
    if isinstance(node, ast.Expr):
        return _eval_node(node.value)
    raise ValueError("Unsupported expression. Use numbers, + - * / // % ** and parentheses only.")

def safe_calc(expr: str) -> float:
    expr = expr.strip()
    if len(expr) > 200:
        raise ValueError("Expression too long.")
    tree = ast.parse(expr, mode="eval")
    return float(_eval_node(tree.body))

@tool
def calculate(expression: str) -> str:
    """Safely evaluate arithmetic like '(12345*6789)+98765'. Returns only the number."""
    try:
        val = safe_calc(expression)
        if abs(val) > 1e18:
            return "Error: result too large."
        # Pretty output: drop trailing .0 if it's an integer
        return str(int(val)) if val.is_integer() else str(val)
    except Exception as e:
        return f"Error: {e}"

print(" Calculator agent ready.")

 Calculator agent ready.


In [70]:
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent

def build_calc_agent():
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0) # Note: Use 1.5-flash or pro
    tools = [calculate]

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a precise math assistant. Use the calculate tool for all math."),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    # create_tool_calling_agent is compatible with ChatPromptTemplate and MessagesPlaceholder
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)

calc_agent1 = build_calc_agent()

In [74]:
print(calc_agent1.invoke({"input": "Compute (12345*6789)+98765. Give only the number."})["output"])

83908970


In [76]:
# Plain LLM (no tool) for comparison
llm_plain = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

q1 = "What is 234 * 19? Give only the number."
q2 = "Compute (12345*6789)+98765. Give only the number."
q3 = "What is 13.7% of 12345 plus 98.76^2? Round to 2 decimals."

print("— LLM only —")
print("Q1:", llm_plain.invoke(q1).content)
print("Q2:", llm_plain.invoke(q2).content)
print("Q3:", llm_plain.invoke(q3).content)

print("\n— Agent with calculator —")
print("Q1:", calc_agent1.invoke({"input": q1})["output"])
print("Q2:", calc_agent1.invoke({"input": q2})["output"])
print("Q3:", calc_agent1.invoke({"input": q3})["output"])

— LLM only —
Q1: 4446
Q2: 83908970
Q3: Let's break this down into two parts and then add them.

**Part 1: 13.7% of 12345**
To calculate a percentage of a number, convert the percentage to a decimal (divide by 100) and then multiply.
0.137 * 12345 = 1691.265

**Part 2: 98.76^2**
This means 98.76 multiplied by itself.
98.76 * 98.76 = 9753.5376

**Add the two parts together:**
1691.265 + 9753.5376 = 11444.8026

**Round to 2 decimals:**
The third decimal place is 2, which is less than 5, so we round down (keep the second decimal place as it is).
11444.80

So, 13.7% of 12345 plus 98.76^2 is **11444.80**.

— Agent with calculator —
Q1: 4446
Q2: 83908970
Q3: [{'type': 'text', 'text': '11444.80', 'extras': {'signature': 'CiQBvj72+9dTBHGuosMuQg2GbUBmlJjBHO5+oOuZne/2eQl0SHQKSQG+Pvb7moybU8SzvYhoBSbW9ZxSdhybK2PqlLzPK8dkn+DmRlb3pJ0nKt+n3ZJJ1PBwPcRJcDVEjjfXY1un02LkThfJtIuyi3QKswEBvj72+0Gam1ffQeRkPfjPhJajWqpNt1V3Kbk4DP8fEOSn76tzxKWwH3chWx7XLL2d+I+mVEDvC6YXeRMA6x9a2DpAZUenowesZCaSISWW9+4yE74qN8T0OLXoQ